# Loops #

In this script, time-series data is extracted from csv files, then combined to a plot.

Configure settings in [setup/config.yaml](../setup/config.yaml). Cf. [template-config.yaml](../templates/template-config.yaml) for an example.

In [ ]:
'''FORCE AUTORELOAD IMPORTS'''
# see https://ipython.org/ipython-doc/3/config/extensions/autoreload.html
%load_ext autoreload
%autoreload 2

In [ ]:
'''IMPORTS'''
import os
import sys

# NOTE: use this if the above reload commands do not work.
for key in list(sys.modules.keys()):
    if key.startswith('src.'):
        del sys.modules[key]

os.chdir(os.path.dirname(_dh[0]))
sys.path.insert(0, os.getcwd())

from src.thirdparty.render import *

from src.setup import config
from src.steps import *

In [ ]:
'''SETUP'''
PATH_USER_CONFIG = 'setup/config.yaml'
config.set_user_config(PATH_USER_CONFIG);

In [ ]:
'''CHOOSE A CASE'''
# NOTE: Instead of this, one can loop over all cases.
case = config.CASES[0]

In [ ]:
'''READ AND PREFORMAT DATA'''
data_p = step_read_data(case.data.pressure, quantity='pressure')
data_p = step_normalise_data(case, data_p, quantity='pressure')
data_v = step_read_data(case.data.volume, quantity='volume')
data_v = step_normalise_data(case, data_v, quantity='volume')

In [ ]:
'''PROCESS DATA'''
# NOTE: determine cycles based on pressure-peaks
data_p = step_recognise_cycles(case, data_p, quantity='pressure', shift='peak')
if case.process.cycles.remove_bad:
    data_p = step_removed_marked_sections(case, data_p)

data_v = step_recognise_cycles(case, data_v, quantity='volume', shift='trough')
if case.process.cycles.remove_bad:
    data_v = step_removed_marked_sections(case, data_v)

In [ ]:
'''FIT CURVE'''
data_p, fitinfos_p = step_fit_curve(case, data_p, quantity='pressure')
data_v, fitinfos_v = step_fit_curve(case, data_v, quantity='volume')

In [ ]:
'''CLASSIFY POINTS'''
_, points_p = step_recognise_points(case, data_p, fitinfos_p, quantity='pressure')
_, points_v = step_recognise_points(case, data_v, fitinfos_v, quantity='volume')

In [ ]:
'''OUTPUT TABLES'''
step_output_single_table(case, data_p, quantity='pressure')
step_output_single_table(case, data_v, quantity='volume')

In [ ]:
'''OUTPUT TIME PLOTS'''
plt = step_output_time_plot_ideal(
    case, data_p, fitinfos_p, points_p, quantity='pressure', symb='P'
)
display(plt)
plt = step_output_time_plot_ideal(
    case, data_v, fitinfos_v, points_v, quantity='volume', symb='V'
)
display(plt)

In [ ]:
'''OUTPUT LOOP PLOTS'''
# TODO